# 内容
```meeting_0122_0_0```の分類版。

# したいこと
```meeting_0122_0_0```の```sklearn.feature_selection.SelectKBest```において```chi2```(χ2)を使う

# 訂正


~~~python
def Montecarlo(T, num_select, MCMC_step_each, E_first, y):
    E = []
    E.append(E_first) 
    for l in range(MCMC_step_each):
        #print(j)
        j = random.randrange(num_feature)
        ori_yi = y[j]
        #print(y[j])
        y[j] = y_renewal(y[j])
        #print(y)
        mask_new = selection(num_select, X, y)
        #print(mask_new)
        E_now = selected_X_energy - each_energy(mask_new)
        #print(E_now)
        E_last = E[-1]
        r = math.exp(-(E_now/T-E_last/T)) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[j] = ori_yi #元に戻す
        else:
            E.append(E_now)
            if E_now ==0:
                E0_Y_lists_df[len(E0_Y_lists_df.columns)] = y
    return E, y #エネルギーのリスト #最後のyだけ(毎回更新しているので)
~~~

このセルにおいて、```j==l``` としてしまっていることに気づいた。
なので上記のように直した。（回帰版も同様に直した）

# 結果
分類を採用したことで```meeting_0122_0_0```よりも多くのサンプルが出てきた気がする、あと速い

➡```num_select = 3, MCMC_step = 100, MCMC_step_each = 5, Max_temper = 500```のとき分類だと5通り、回帰だと2通り

# 展望
* 時期関係なく全部の```energy==0```を集める　➡実行
* 全部の```energy==0```となる通りを分類と回帰で比較

# 出力
* 各温度におけるエネルギーの推移
* 各温度での、全部のステップを経た後のy
* 各温度における最後の実験で、エネルギーが0となったものだけ表示されるdataframe
* 温度関係なく、エネルギーが0となったすべての```y```を格納したdataframe　⬅追加

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, chi2
import pprint

In [2]:
#valiable
num_select = 3

MCMC_step = 1000
MCMC_step_each = 20

Max_temper = 5000
T = [j for j in range(1, Max_temper+1)]
len_T = len(T)

b = [1/ l for l in T]

In [3]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）


data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
#print(data_yields)
ori_y = data_yields.T.iloc[0]
print(ori_y)

y= data_yields.T.iloc[0]

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64


In [4]:
type(X) #pandas.core.frame.DataFrame
X.shape #(20, 20)

type(y) #pandas.core.series.Series
y.shape #(20,)

type(y[0]) #numpy.int64

type(data_yields) #pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

In [5]:
#特徴量の分散や同じ特徴量が存在するのかを確認

from sklearn.feature_selection import VarianceThreshold
select_VT = VarianceThreshold(threshold=0)
select_VT.fit(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(select_VT.get_support())) #20、すなわち同じ特徴量は存在していないことがわかった

20


In [6]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    select_SKB = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #chi2...分類
    select_SKB.fit(X, y)
    mask_each = select_SKB.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask_each

In [7]:
def each_energy(mask_each): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [8]:
def y_renewal(s: int):
    return 1-s

In [9]:
E0_Y_lists_df = pd.DataFrame()
E0_Y_lists_df[0]=y #初期設定のyを追加

In [10]:
def Montecarlo(T, num_select, MCMC_step_each, E_first, y):
    E = []
    E.append(E_first)
    
    for l in range(MCMC_step_each):
        #print(j)
        j = random.randrange(num_feature)
        ori_yi = y[j]
        #print(y[j])
        y[j] = y_renewal(y[j])
        #print(y)
        mask_new = selection(num_select, X, y)
        #print(mask_new)
        E_now = selected_X_energy - each_energy(mask_new)
        #print(E_now)
        E_last = E[-1]
        r = math.exp(-(E_now/T-E_last/T)) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[j] = ori_yi #元に戻す
        else:
            E.append(E_now)
            if E_now ==0:
                E0_Y_lists_df[len(E0_Y_lists_df.columns)] = y
    return E, y #エネルギーのリスト #最後のyだけ(毎回更新しているので)

In [11]:
E_lists = []
Y_lists = []

mask_ori = selection(num_select, X, ori_y)
print(mask_ori)
selected_X_energy = each_energy(mask_ori)
ori_y = data_yields.T.iloc[0]

[False False False False False  True False False False False False  True
 False False False False False False False  True]


In [12]:
for k in range(len_T):
    E, y = Montecarlo(T[k], num_select, MCMC_step_each, 0, ori_y) #0=each_energy(mask_ori, y)-each_energy(mask_ori, y)
    #print(y)
    E_lists.append(E) #はじめのE=0は入れていないぞ
    Y_lists.append(y)
    data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
    ori_y = data_yields.T.iloc[0]
    #y = data_yields.T.iloc[0] #yの初期化
    #print(Y_lists[-1])
    #print(Y_lists[-1]==y)

In [13]:
#Y_lists[0]==ori_y

In [14]:
#Y_lists[1]

In [15]:
Y_lists_df_middle = pd.DataFrame()
Y_lists_df_middle.index.name = 'y'
Y_lists_df_middle.columns.name = 'temperature'

for t in range(len_T):
    Y_lists_df_middle[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df_middle['ori']=ori_y
Y_lists_df_middle

temperature,1,2,3,4,5,6,7,8,9,10,...,4992,4993,4994,4995,4996,4997,4998,4999,5000,ori
0,1,1,1,1,0,0,0,1,1,0,...,1,1,1,1,1,0,1,0,0,1
1,1,1,0,0,0,1,1,1,0,0,...,1,0,1,0,0,0,1,0,0,0
2,0,0,1,1,1,0,0,0,1,0,...,1,0,0,1,0,1,0,0,1,0
3,1,0,0,1,1,1,0,0,1,1,...,1,1,0,1,1,1,0,0,0,1
4,0,1,1,1,0,0,1,1,1,1,...,1,1,1,0,0,1,1,1,1,1
5,1,0,0,0,1,0,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0
6,1,0,1,0,0,1,0,1,1,1,...,1,0,1,0,0,1,0,0,1,0
7,1,1,0,1,1,0,0,1,0,0,...,1,0,1,0,0,0,0,1,1,1
8,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
9,0,0,0,1,0,0,0,1,1,1,...,1,0,1,0,1,0,0,1,0,1


In [16]:
E_lists_df = pd.DataFrame(index=[], columns=[])

#del E_lists[0]#元の条件削除⇒len(E_lists)=MCMC_step_each となるはず

i = random.randrange(len_T-1)
#r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
rnd = np.random.uniform(0.0, 1.0)
if rnd < r:
    E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]

In [17]:
Y_lists_df_middle = pd.DataFrame()
Y_lists_df_middle.index.name = 'y'
Y_lists_df_middle.columns.name = 'temperature'

for t in range(len_T):
    Y_lists_df_middle[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df_middle['ori']=ori_y
Y_lists_df_middle

temperature,1,2,3,4,5,6,7,8,9,10,...,4992,4993,4994,4995,4996,4997,4998,4999,5000,ori
0,1,1,1,1,0,0,0,1,1,0,...,1,1,1,1,1,0,1,0,0,1
1,1,1,0,0,0,1,1,1,0,0,...,1,0,1,0,0,0,1,0,0,0
2,0,0,1,1,1,0,0,0,1,0,...,1,0,0,1,0,1,0,0,1,0
3,1,0,0,1,1,1,0,0,1,1,...,1,1,0,1,1,1,0,0,0,1
4,0,1,1,1,0,0,1,1,1,1,...,1,1,1,0,0,1,1,1,1,1
5,1,0,0,0,1,0,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0
6,1,0,1,0,0,1,0,1,1,1,...,1,0,1,0,0,1,0,0,1,0
7,1,1,0,1,1,0,0,1,0,0,...,1,0,1,0,0,0,0,1,1,1
8,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
9,0,0,0,1,0,0,0,1,1,1,...,1,0,1,0,1,0,0,1,0,1


In [18]:
#data_yields.T.iloc[0]

In [19]:
for j in range(int(MCMC_step/MCMC_step_each)):

    for k in range(len_T):
        E, y = Montecarlo(T[k], num_select, MCMC_step_each, E_lists[k][-1], Y_lists[k])
        E_lists.append(E)
        Y_lists.append(y)

    E_lists_df[j] = E_lists[0:len_T]
    del E_lists[0:len_T]#1つ前のループで作られたリストの削除
    del Y_lists[0:len_T]

    i = random.randrange(len_T-1)
    #r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
    r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
    rnd = np.random.uniform(0.0, 1.0)
    if rnd < r:
        E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]


#各温度でMCMC_step回動作させたことで、{MCMC_step_each個のエネルギーと最終y}のデータがkセットできた
#最後に知りたいのは、最終ykセットと、エネルギーの動き
E_lists_df.index.name = 'temperature'
E_lists_df.columns.name = 'each MCMC step'
E_lists_df

each MCMC step,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
temperature,,,,,,,,,,,,,,,,,,,,,
0,"[0, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[4, 4, 4, 4, 6, 4, 4, 4, 4, 4]","[4, 4, 4, 4, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2]","[2, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 2, 2, 4, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2]","[2, 2, 2, 2, 4, 2, 4, 4, 4, 4, 4, 4]","[4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2]","[2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 0]","[0, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 2, 2, 2]",...,"[4, 4, 4, 4, 2, 2, 2, 2, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, ...","[4, 4, 4, 4, 2, 4, 4, 4, 2]","[2, 2, 2, 2, 4, 6, 4, 6, 6, 6, 6, 4, 4, 4, 4, ...","[2, 2, 2, 2, 4, 4, 4, 2, 4, 4, 4, 4, 2, 2, 2]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[2, 4, 4, 4, 2, 4, 4, 4, 2, 2, 4, 4, 0]","[0, 0, 2, 0, 0, 2, 2]","[2, 2, 2, 0, 0]","[0, 2, 2, 2, 2, 2, 2, 2, 4, 4]"
1,"[0, 2, 2, 4, 4, 6, 6, 6, 6, 6, 6, 4, 4, 4]","[4, 4, 2, 2, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 6, 6, 2, 2, 4, 6, 6, 6, 6, 6, ...","[4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2]","[2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2]","[2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 2, 0, 2, 4, 2, 4]","[4, 2, 2, 4, 4, 4, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 6, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 6, ...",...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, ...","[6, 6, 6, 6, 6, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, 4, 4, 4, ...","[4, 6, 6, 4, 6, 4, 4, 4, 2, 2, 2, 4, 6, 6, 6, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, ...","[6, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 4, 4, 4, ...","[4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 6, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 6, 6, 4, ...","[4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 6]","[6, 4, 6, 6, 6, 6, 6, 6, 4, 6, 4, 4, 4, 6, 6, ..."
2,"[0, 2, 2, 2, 2, 2, 4, 4, 6, 6, 4, 6, 4, 4, 2, 4]","[4, 2, 6, 2, 4, 0, 0, 2, 2, 4, 4, 2, 6, 4, 4, ...","[6, 6, 6, 6, 2, 2, 2, 2, 2, 2]","[2, 4, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2]","[2, 4, 4, 4, 4, 4, 6, 6, 6, 4, 4, 4, 4, 4, 6, ...","[6, 6, 4, 4, 6, 6, 4, 4, 4, 4, 4, 4, 2, 4, 2, ...","[4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, ...","[6, 6, 6, 6, 6, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4]",...,"[4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, ...","[4, 4, 6, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 2]","[2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 2, 4, 4, ...","[6, 4, 6, 4, 4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 2]","[2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 4, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, 4, 4, 6, ...","[4, 6, 6, 6, 6, 6, 4, 4, 6, 6, 6, 6, 6, 4, 4, ...","[2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4]","[4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, ...","[4, 4, 4, 2, 2, 2, 2, 4, 6, 6, 4, 4, 4, 4, 4, ..."
3,"[0, 0, 2, 4, 4, 4, 2, 2, 2, 4, 4, 2, 4, 4, 4, ...","[2, 6, 4, 6, 6, 6, 4, 4, 2, 6, 4, 2, 4, 4, 4, ...","[4, 4, 4, 4, 2, 4, 6, 6, 4, 4, 4, 4, 6, 6, 6, ...","[6, 6, 4, 4, 4, 4, 4, 4, 4, 6, 4, 6, 4, 4, 4, ...","[4, 4, 6, 6, 4, 6, 6, 6, 4, 4, 4, 4, 6, 2, 4, ...","[2, 4, 4, 2, 4, 2, 4, 6, 6, 6, 6, 6, 6, 6, 6, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[6, 6, 4, 4, 6, 6, 6, 4, 6, 6, 4, 4, 6, 6, 6, ...","[6, 6, 6, 6, 6, 6, 6, 4, 2, 4, 4, 4, 4, 4, 4, ...","[4, 4, 6, 4, 6, 4, 6, 6, 6, 6, 4, 6, 6, 6, 6, ...",...,"[4, 6, 6, 6, 6, 4, 6, 4, 6, 4, 6, 6, 6, 4, 6, ...","[4, 4, 4, 6, 4, 6, 6, 4, 6, 4, 6, 4, 4, 4, 4, ...","[4, 6, 6, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 2, 2, ...","[2, 2, 4, 4, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, ...","[4, 6, 6, 6, 4, 2, 2, 2, 4, 4, 6, 6, 4, 4, 2, ...","[4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 2, 2, ...","[4, 4, 4, 2, 2, 6, 4, 4, 4, 4, 6, 2, 2, 2, 2, 2]","[2, 4, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 6, 6, 4, ...","[4, 4, 4, 2, 4, 4, 4, 4, 2, 2, 2, 2, 4, 2, 2, ..."
4,"[0, 2, 2, 4, 4, 4, 6, 6, 4

In [20]:
Y_lists_df = pd.DataFrame()
Y_lists_df.index.name = 'y'
Y_lists_df.columns.name = 'temperature'

for t in range(len_T):
    Y_lists_df[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df['ori']=ori_y
Y_lists_df

temperature,1,2,3,4,5,6,7,8,9,10,...,4992,4993,4994,4995,4996,4997,4998,4999,5000,ori
0,1,0,0,0,1,0,0,0,1,0,...,1,0,1,0,1,1,1,1,0,1
1,0,0,0,1,1,1,0,0,0,1,...,0,0,0,0,0,1,1,1,1,0
2,0,1,0,0,0,0,1,0,1,0,...,1,1,0,1,0,0,1,1,1,0
3,1,1,1,0,0,1,1,0,0,0,...,0,1,1,1,1,1,0,0,0,1
4,0,1,1,0,0,0,0,0,1,1,...,0,0,0,1,1,0,0,0,1,1
5,1,0,0,0,0,1,1,1,0,1,...,0,0,0,1,1,1,0,1,0,0
6,0,1,0,1,1,0,0,0,1,0,...,1,0,1,1,1,1,0,0,1,0
7,1,1,0,0,1,0,0,1,0,1,...,1,1,1,0,0,1,0,1,0,1
8,1,0,0,1,1,1,1,0,1,1,...,0,1,1,1,1,1,0,0,1,0
9,1,1,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,1


In [21]:
#エネルギーが0となったものyを示したdataframe
E0_Y_lists_df.index.name = 'y selected featrues are same as ori_y'
E0_Y_lists_df.T.drop_duplicates().T 

,0,1,2,3,4,5,6,8,9,11,...,17155,17241,17266,17276,17334,17444,17454,17606,17695,17821
y selected featrues are same as ori_y,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,0,0,1,...,0,1,1,0,0,0,0,1,1,1
1,0,0,0,1,1,0,1,0,0,1,...,1,1,1,0,0,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,1,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,1,1,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,1,0,0
7,1,1,1,1,1,1,1,1,1,1,...,0,0,1,0,1,0,0,0,1,1
8,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,1


In [22]:
print('特徴量を{}個選ぶとすると、解の個数は{}通りとなった（入力のyも含む）'.format(num_select, len(E0_Y_lists_df.T.drop_duplicates().T.columns)))

特徴量を3個選ぶとすると、解の個数は2846通りとなった（入力のyも含む）


In [23]:
#E_lists[len_T - 1][-1] #IndexError: list index out of range
E_lists[0][-1] == 0 #False
E_lists[495][-1] == 0 #False
E_lists[16][-1] == 0 #True(t = 17)

for t in range(len_T):
    if E_lists[t][-1]==0:
        print(t+1) #温度は1度からなので

364
370
734
985
1467
1821
2008
2694
2882
3383
4759
4872


In [24]:
#各温度における最後の実験で、エネルギーが0となったものだけ表示されるようなdataframe
Y_lists_lastE0_df = pd.DataFrame()
Y_lists_lastE0_df.index.name = 'y'
Y_lists_lastE0_df.columns.name = 'temperature'

for t in range(len_T):
    if E_lists[t][-1] == 0: 
        Y_lists_lastE0_df[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_lastE0_df['ori']=ori_y

Y_lists_lastE0_df

temperature,364,370,734,985,1467,1821,2008,2694,2882,3383,4759,4872,ori
0,0,0,1,0,1,0,0,1,1,1,0,1,1
1,1,0,0,1,1,0,0,0,0,1,0,1,0
2,1,0,0,1,1,0,1,0,0,1,0,1,0
3,0,1,1,0,0,1,0,1,1,0,1,0,1
4,0,1,1,0,0,1,0,1,1,0,1,0,1
5,1,1,0,0,0,1,0,1,1,0,1,0,0
6,1,0,0,1,1,0,1,0,0,1,0,0,0
7,0,1,1,0,0,1,0,1,1,0,1,0,1
8,1,1,1,0,1,0,0,0,1,0,0,0,0
9,0,1,1,0,0,1,0,1,1,0,1,0,1
